# Payment Matching
This competition was held by Shopee ID NSDC, 2020.

Source : [https://www.kaggle.com/c/payment-id-nsdc-2020]

### Question and problem understanding
At Shopee, bank transfer is a payment method in most countries. When a buyer chooses to place an order using bank transfer, he/she is supposed to make the transfer within 2 days after he/she places the order.
After he/she makes the transfer, Shopee will receive a bank statement from the bank and Shopee needs to compare and match the bank statement with the checkout information in order to confirm that this particular order has been paid. This process is called payment matching.
Two criteria need to be met in order to match a bank statement with a checkout :
* Amount match : statement amount equals checkout amount.
* Name match : statement description "matches" checkout buyer name (Note: statement description usually contains buyer name).

A proper match occurs when both the amount and the name matches on both bank statement and checkout list.

Based on the two criteria, lets match the bank statement IDs to the checkout IDs. Please note that in the dataset provided, there are unique matches of each bank statement and checkout.

**Importing needed packages**

In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

D:\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


**Importing datasets**

In [2]:
bank_statement = pd.read_csv('bank_statement.csv')
checkout = pd.read_csv('checkout.csv')
print(bank_statement.shape, checkout.shape)

(240000, 3) (240000, 3)


**Data overview**

In [3]:
bank_statement.head(10)

,stmt_id,stmt_amount,desc
0,1,100.00,INSTANT TRANSFER BAW CGANBAWI YIWHI * LEZC ...
1,2,23.77,WOOG CARIZA NIWSI YO
2,3,18.00,ZOMAZAW MABIX ATMA * BUGW * KZBUGW - TRANSFER...
3,4,21.87,TRANSFER GO LA/FA AHHA PUYIMA * YEBF ...
4,5,53.68,INSTANT TRANSFER FABZUN WIQAT JIW FA* TEGF ...
5,6,27.70,DZZ RL- OBS ZAO CI * SHOPE...
6,7,9.19,SOYEGQAJR SOYEGQAJR PIFI ULAMAX
7,8,27.33,HAMIVASIAVMI PIVMI * SHOPEE * FJ - TRANSFERT...
8,9,23.30,SHOPEE NUJUX NAZAMHA CINZI
9,10,13.00,HRR CQ- YIEL CJI POB * ZM * ...


In [4]:
checkout.head(10)

,ckt_id,ckt_amount,buyer_name
0,1,20.00,SAVMUEJICE EPHOCP
1,2,14.91,LUHALLAK ZGANFAP HAIQAW VIP LOHK. CIQTI
2,3,13.20,SUQ UNAISIE
3,4,36.90,GOZK BIHKAUT LIV LATAPI
4,7,26.29,HUY ZABIRAN HATIZAN WIHMI IRYIZ
5,8,23.94,WIL ZOOBP MEBP
6,10,62.73,RFEVQES JUASJ ASAB ANAS
7,12,102.50,PAQZICIA ZOLE YOMISIC CVAS
8,13,25.40,GANIBAG
9,15,9.77,ATIYUV WAUNIF


**Removing special characters**

We will remove special characters and "TRANSFER" from the buyer_name and desc columns.

In [5]:
bank_statement.desc = bank_statement.desc.apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
bank_statement.desc = bank_statement.desc.apply(lambda x: re.sub(r"TRANSFER", '', x))
checkout.buyer_name = checkout.buyer_name.apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
checkout.buyer_name = checkout.buyer_name.apply(lambda x: re.sub(r"TRANSFER", '', x))

**Sorting amount by value**

In [6]:
bank_statement = bank_statement.sort_values(by=['stmt_amount'])
checkout = checkout.sort_values(by=['ckt_amount'])

**Converting the desc and buyer_name to SET**

In [7]:
bank_statement.desc = bank_statement.desc.apply(lambda x: set(x.split()))
checkout.buyer_name = checkout.buyer_name.apply(lambda x: set(x.split()))

In [8]:
bank_statement.head(10)

,stmt_id,stmt_amount,desc
37740,37741,0.01,"{BUDABBSHOPEEAW, ZAUGAH, ZIFY}"
150536,150537,0.01,"{PIHK, TO, A, TYA, INSTANT, PAHK, C, PGGEPC, A..."
226724,226725,0.01,"{PAQB, WOK, PEE}"
230963,230964,0.01,"{FAWXANAH, VTVUQINR, GOFONR, VUQINR, WAQIN, NOL}"
115857,115858,0.01,"{TO, CLA, BULVIBJ, A, NG, C, SHOPEE, ACP, IBAX..."
57353,57354,0.01,"{HIMHAW, SIZ, AWNAMH, SAJTUY, I, NAMH}"
39780,39781,0.01,"{XAI, TO, PRXAI, A, MIKSI, C, KOOVAWYIXIK, GIK}"
25601,25602,0.01,"{PUWAPPAM, A, INSTANT, ASLAY, SHOPEEPOBIGEPAGA..."
127225,127226,0.01,"{APACY, SUCF, PICRI, QOVCI, MWSUCF}"
169611,169612,0.01,"{AFHUK, GAYZETD, FIT, GZGAYZETD, ZIQ, VAHIX}"


In [9]:
checkout.head(10)

,ckt_id,ckt_amount,buyer_name
77722,106443,0.01,"{CIN, RAXVAN, WXAKIH}"
69694,95499,0.01,"{DUX, RAMAXADI, ZIDGI, RUWVICWI}"
35591,48817,0.01,"{IBAXILAY, ACP, LAYGAB}"
133597,183132,0.01,"{ZIFYI, XIH, ZAUGAH, AXIWIH, BUDABBAW}"
167547,229799,0.01,"{PIHK, PAHK, AI}"
205619,281850,0.01,"{NAAQAT, QADBIBA, XUT, WIXDI}"
199175,273052,0.01,"{ZEHITOSO, SASASQ}"
217898,298686,0.01,"{XAM, WARIYRILDI, JOWAJAB}"
208502,285819,0.01,"{AKIQAZ, IHABI, LIBKI, BUV}"
1764,2386,0.01,"{GATIBAB, QIGYI, WACLI, VOBC}"


**Matching price and description**

Criteria 1 : amount match!

In [10]:
# Sorted list of prices
prices = sorted(list(set(checkout.ckt_amount)))

In [11]:
# Matching the price and description
bs, co = bank_statement.values.tolist(), checkout.values.tolist(), 
curr_bank, curr_trans, pend_bank, pend_trans = [], [], [], []
answer = []
no_ans_trans, no_ans_bank = [], []
for price in prices:
    while bs and bs[0][1] == price:
        curr_bank.append(bs.pop(0))
    while co and co[0][1] == price:
        curr_trans.append(co.pop(0))

    for trans in curr_trans:
        found = False
        curname = trans[2]
        bk = list(filter(lambda x: x[2].intersection(curname), curr_bank))
        if bk:
            b = max(bk, key= lambda x: len(x[2].intersection(curname)))
            answer.append((trans, b))
            curr_bank.remove(b)
        else:
            no_ans_trans.append(trans)
            
    no_ans_bank.extend(curr_bank)
            
    curr_bank, curr_trans = [], []

In [12]:
# Number of transaction that does not match
print(len(no_ans_bank), len(no_ans_trans))

3143 3143


In [13]:
no_ans_bank[:5]

[[134561, 0.1, {'AYRIHAW', 'NIMPI', 'SRZB', 'YUQOM', 'ZB'}],
 [122394, 0.1, {'A', 'C', 'GQ', 'PUJPANIFA', 'SHOPEE', 'TO', 'WAJFAPA'}],
 [158581, 0.1, {'A', 'AD', 'ALAZJAJHICA', 'DILGI', 'GO', 'NA', 'ZA'}],
 [9325,
  0.11,
  {'A', 'B', 'BAIC', 'BJTROB', 'C', 'PALA', 'TO', 'TROB', 'V', 'ZAYTRIJI'}],
 [145415, 0.11, {'A', 'DA', 'JA', 'KULBAGIFAB', 'VO'}]]

In [14]:
no_ans_trans[:5]

[[309643, 0.1, {'ALA', 'ZJAJHICA'}],
 [275938, 0.1, {'PANIFA', 'PUJ', 'WAJFAPPA'}],
 [117315, 0.1, {'AYRIHAWYUQOMB'}],
 [141553, 0.11, {'BAGIFAB'}],
 [8350, 0.11, {'PALAZAYTRJI'}]]

## Fuzzy Search (for transaction that does not match)

In computer science, fuzzy string matching is the technique of finding strings that match a pattern approximately (rather than exactly). In another word, fuzzy string matching is a type of search that will find matches even when users misspell words or enter only partial words for the search.

In [15]:
# Getting IDs of transaction that does not match
no_id_bank, no_id_trans = [], []
for x in range(len(no_ans_bank)):
    no_id_bank.append(no_ans_bank[x][0])
    no_id_trans.append(no_ans_trans[x][0])

In [16]:
# Getting subset of dataframe that previously has no match
checkout2 = checkout[checkout['ckt_id'].isin(no_id_trans)]
bank_statement2 = bank_statement[bank_statement['stmt_id'].isin(no_id_bank)]

In [17]:
bank_statement2.head()

,stmt_id,stmt_amount,desc
134560,134561,0.10,"{SRZB, NIMPI, YUQOM, AYRIHAW, ZB}"
122393,122394,0.10,"{TO, A, C, PUJPANIFA, SHOPEE, GQ, WAJFAPA}"
158580,158581,0.10,"{ZA, A, NA, GO, AD, ALAZJAJHICA, DILGI}"
9324,9325,0.11,"{TO, A, V, C, TROB, ZAYTRIJI, BAIC, B, PALA, B..."
145414,145415,0.11,"{A, DA, KULBAGIFAB, JA, VO}"


In [18]:
checkout2.head()

,ckt_id,ckt_amount,buyer_name
225903,309643,0.10,"{ZJAJHICA, ALA}"
201280,275938,0.10,"{PANIFA, WAJFAPPA, PUJ}"
85648,117315,0.10,{AYRIHAWYUQOMB}
103356,141553,0.11,{BAGIFAB}
6114,8350,0.11,{PALAZAYTRJI}


In [19]:
# Converting set to string
bank_statement2['desc'] = bank_statement2['desc'].apply(lambda x: ' '.join(sorted(list(x))))
checkout2['buyer_name'] = checkout2['buyer_name'].apply(lambda x: ' '.join(sorted(list(x))))

<ipython-input-19-6eeba9652afe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_statement2['desc'] = bank_statement2['desc'].apply(lambda x: ' '.join(sorted(list(x))))
<ipython-input-19-6eeba9652afe>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkout2['buyer_name'] = checkout2['buyer_name'].apply(lambda x: ' '.join(sorted(list(x))))


In [20]:
bank_statement2.head()

,stmt_id,stmt_amount,desc
134560,134561,0.10,AYRIHAW NIMPI SRZB YUQOM ZB
122393,122394,0.10,A C GQ PUJPANIFA SHOPEE TO WAJFAPA
158580,158581,0.10,A AD ALAZJAJHICA DILGI GO NA ZA
9324,9325,0.11,A B BAIC BJTROB C PALA TO TROB V ZAYTRIJI
145414,145415,0.11,A DA JA KULBAGIFAB VO


In [21]:
checkout2.head()

,ckt_id,ckt_amount,buyer_name
225903,309643,0.10,ALA ZJAJHICA
201280,275938,0.10,PANIFA PUJ WAJFAPPA
85648,117315,0.10,AYRIHAWYUQOMB
103356,141553,0.11,BAGIFAB
6114,8350,0.11,PALAZAYTRJI


In [22]:
prices = sorted(list(set(checkout2.ckt_amount)))

**Applying Fuzzy Search**

Criteria 2 : name match!

In this case, we will use fuzzy search to match the buyer_name with description from bank statement.

In [23]:
bs2, co2 = bank_statement2.values.tolist(), checkout2.values.tolist(), 
curr_bank2, curr_trans2 = [], []
answer2 = []
for price in prices:
    while bs2 and bs2[0][1] == price:
        curr_bank2.append(bs2.pop(0))
    while co2 and co2[0][1] == price:
        curr_trans2.append(co2.pop(0))
    for trans in curr_trans2:
        curname = trans[2]
        b = max(curr_bank2, key= lambda x: fuzz.partial_ratio(curname, x[2]))
        answer2.append((trans, b))
        curr_bank2.remove(b)
    curr_bank2, curr_trans2 = [], []

**Construct final dataframe**

Lets match the bank statement IDs to the checkout IDs!

In [24]:
df1 = [(x[0][0], x[1][0]) for x in answer]
df2 = [(x[0][0], x[1][0]) for x in answer2]
df = pd.DataFrame(df1+df2)
df.columns = ['ckt_id', 'stmt_id']
df = df[['stmt_id', 'ckt_id']]

In [25]:
print(df.shape)

(240000, 2)


In [26]:
df.head()

,stmt_id,ckt_id
0,114360,106443
1,201905,95499
2,115858,48817
3,37741,183132
4,150537,229799


In [28]:
submission = pd.DataFrame(answer2).to_csv('my_submission.csv', index=False)

References :

[https://www.kaggle.com/matheusaaron/payment-matching-by-cupu123]

[https://towardsdatascience.com/natural-language-processing-for-fuzzy-string-matching-with-python-6632b7824c49#:~:text=In%20computer%20science%2C%20fuzzy%20string,partial%20words%20for%20the%20search.]